# Optimizing Computer Vision Applications

This tutorial shows some techniques to get better performance for computer vision applications with the Intel® Distribution of OpenVINO™ toolkit.


## 1. Setup the environment variables,download model files and import dependencies

In [1]:
from IPython.display import HTML
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent.parent))
from demoTools.demoutils import *

In [2]:
!/opt/intel/openvino/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


In [3]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name ssd300 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name ssd512 -o models

################|| Downloading models ||################

========== Downloading models/public/mobilenet-ssd/mobilenet-ssd.prototxt
... 100%, 28 KB, 59858 KB/s, 0 seconds passed

========== Downloading models/public/mobilenet-ssd/mobilenet-ssd.caffemodel
... 100%, 22605 KB, 22613 KB/s, 0 seconds passed

################|| Post-processing ||################

################|| Downloading models ||################

========== Downloading models/public/ssd300/ssd300.tar.gz
... 100%, 95497 KB, 27135 KB/s, 3 seconds passed

################|| Post-processing ||################

========== Unpacking models/public/ssd300/ssd300.tar.gz
========== Replacing text in models/public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft/deploy.prototxt
################|| Downloading models ||################

========== Downloading models/public/ssd512/ssd512.tar.gz
... 100%, 98624 KB, 24112 KB/s, 4 seconds passed

################|| Post-processing ||################

========== Unpacking models/public/

In [4]:
!mkdir -p models/object_detection/SSD512/{FP16,FP32} 
!mkdir -p models/object_detection/SSD300/{FP16,FP32}

### Run Model Optimizer on the models to get IR files

First, we will create the required directories, then run the model Optimizer to get the IR files. 

In [5]:
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel -o models/object_detection/common/mobilenet-ssd/FP32/ --scale 256 --mean_values [127,127,127]
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel -o models/object_detection/common/mobilenet-ssd/FP16/ --scale 256 --mean_values [127,127,127] --data_type FP16
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/object_detection/common/ssd/300/caffe/ssd300.caffemodel -o models/object_detection/SSD300/FP32/
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/object_detection/common/ssd/300/caffe/ssd300.caffemodel -o models/object_detection/SSD300/FP16/ --data_type FP16
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/object_detection/common/ssd/512/caffe/ssd512.caffemodel -o models/object_detection/SSD512/FP32/
! python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_caffe.py --input_model models/object_detection/common/ssd/512/caffe/ssd512.caffemodel -o models/object_detection/SSD512/FP16/ --data_type FP16

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/optimization-tools-and-techniques/devcloud/cpp/models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/optimization-tools-and-techniques/devcloud/cpp/models/object_detection/common/mobilenet-ssd/FP32/
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	Fals


[ SUCCESS ] Generated IR model.
[ SUCCESS ] XML file: /home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/optimization-tools-and-techniques/devcloud/cpp/models/object_detection/SSD512/FP32/ssd512.xml
[ SUCCESS ] BIN file: /home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/optimization-tools-and-techniques/devcloud/cpp/models/object_detection/SSD512/FP32/ssd512.bin
[ SUCCESS ] Total execution time: 9.98 seconds. 
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/optimization-tools-and-techniques/devcloud/cpp/models/object_detection/common/ssd/512/caffe/ssd512.caffemodel
	- Path for generated IR: 	/home/u30239/Smart_video_review/iot-devcloud/smart-video-workshop/optimization-tools-and-techniques/devcloud/cpp/models/object_detection/SSD512/FP16/
	- IR output name: 	ssd512
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not speci

## 2. Pick the right model based on application and hardware

Use/train a model with the right performance/accuracy tradeoffs. Performance differences between models can be bigger than any optimization you can do at the inference app level. Run various SSD models from the model_downloader in the car detection example which we used in the initial tutorial and observe the performance. We will run these tests on different hardware accelerators to determine how application performance depends on models as well as hardware.

In the previous step we have all the models convered and ready by model Optimizer. 

### Set environmental variables

In [6]:
!ln -sf /data/reference-sample-data/object-detection-python/cars_1900.mp4 
videoHTML('Cars video', ['cars_1900.mp4'])

### Compile the code

The code in this demo is separated into two parts.
First part is responsible for reading the input stream and running the object detection inference workload on the stream. 
This part outputs Region Of Interest (ROI), in terms of coordinates, for each frame.
The source code for this part can be found in [main.cpp](./main.cpp), and the executable will be named "tutorial1".
Output ROI will be written into a text file, "ROIs.txt".

The second part reads the ROIs.txt file, and overlays boxes on each frame of the stream based on the coordinates.
Then the output video is written into a file. 
The source code for this step is in [ROI_writer.cpp](./ROI_writer.cpp).

We have provided a Makefile for compiling the examples. Run the following cell to compile the application.
(tip: use **crtl+enter** to run the cell)

In [7]:
!make

make: Nothing to be done for 'all'.


### Commandline flags

The two executables, tutorial1 and ROIwriter, take a number of commandline arguments.

Run the following cells to see the list of the available arguments: 

In [8]:
! ./tutorial1 -h


[usage]
	tutorial1 [option]
	options:

		-h              Print a usage message
		-i <path>       Required. Path to input video file
		-model <path>   Required. Path to model file.
		-b #            Batch size.
		-thresh #       Threshold (0-1: .5=50%)
		-d <device>     Infer target device (CPU or GPU or MYRIAD)
		-fr #           maximum frames to process
		-o #            Path to the output ROIs file


In [9]:
!./ROI_writer -h


[usage]
	ROIviewer [option]
	options:

		-h              Print a usage message
		-i <path>       Required. Path to input video file
		-ROIfile <path> Path to ROI file.
		-b #            Batch # to display.
		-l <path>       class labels file.
		-o <filename>       Output file path.
		-r <res>       (double) factor to cut reolution by; 2. will cut the resolution by half.
		-k <keepframe>       Writer will keep every <skipframe> frames.


### Create Job Script 

We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-m`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the optimized models XML
* `-i`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-r`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory
* `-d`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;hardware device type (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* `-n`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;number of infer requests

The job file will be executed directly on the edge compute node.

In [10]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
OUTPUT_FILE=$1
DEVICE=$2
MODEL=$3
# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE
ROIFILE=$OUTPUT_FILE/ROIs.txt
OVIDEO=$OUTPUT_FILE/output.mp4

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_bitstreams/2019R1_PL1_FP11_MobileNetCaffe.aocx 
fi

if [ "$MODEL" = "FP32" ]; then
    config_file="conf_fp32.txt"
else
    config_file="conf_fp16.txt"
fi

# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
./tutorial1 -i /data/reference-sample-data/object-detection-python/cars_1900.mp4 \
            -m models/object_detection/$MODEL \
            -d $DEVICE \
            -o $OUTPUT_FILE\
            -fr 3000

# Converting the text output to a video
./ROI_writer -i /data/reference-sample-data/object-detection-python/cars_1900.mp4 \
             -o $OUTPUT_FILE \
             -ROIfile $ROIFILE \
             -l pascal_voc_classes.txt \
             -r 2.0 # output in half res

Overwriting object_detection_job.sh


## 3. Run the object detection example with different models on different devices.

For simplicity of the code and in order to put more focus on the performance number, video rendering with rectangle boxes for detected objects has been separated from object detection example(tutorial1.py). The inference difference in different scenarios can be seen in the progress bar after running the sample. 


### a) CPU

#### - Inferencing using **mobilenet-ssd** model

In [11]:
print("Submitting a job to an edge compute node with an Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te -F "results/core/mobilenet CPU common/mobilenet-ssd/FP32/mobilenet-ssd.xml" -N obj_det_cpu
print(job_id_core[0])
#Progress indicators
if job_id_core:
   progressIndicator('results/core/mobilenet', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
   progressIndicator('results/core/mobilenet', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

Submitting a job to an edge compute node with an Intel Core CPU...
4980.v-qsvr-1.devcloud-edge


#### - Inferencing using **ssd300** model

In [6]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:i5-6500te -F "results/Core/ssd300 CPU SSD300/FP32/ssd300.xml" -N obj_det_cpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/ssd300', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/Core/ssd300', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4747.v-qsvr-1.devcloud-edge


#### - Inferencing using **ssd512** model

In [7]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:i5-6500te -F "results/Core/ssd512 CPU SSD512/FP32/ssd512.xml" -N obj_det_cpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/ssd512', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/Core/ssd512', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4748.v-qsvr-1.devcloud-edge


### b) GPU

#### - Inferencing using **mobilenet-ssd** model

In [8]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/Core/mobilenet GPU common/mobilenet-ssd/FP32/mobilenet-ssd.xml" -N obj_det_gpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/mobilenet', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/Core/mobilenet', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4749.v-qsvr-1.devcloud-edge


#### - Inferencing using model: ssd300

In [9]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/Core/ssd300 GPU SSD300/FP32/ssd300.xml" -N obj_det_gpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/ssd300', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/Core/ssd300', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4750.v-qsvr-1.devcloud-edge


#### - Inferencing using model: ssd512

In [10]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/Core/ssd512 GPU SSD512/FP32/ssd512.xml" -N obj_det_gpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/Core/ssd512', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/Core/ssd512', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4751.v-qsvr-1.devcloud-edge


### c) Intel® Movidius™ Neural Compute Stick

#### - Inferencing using **mobilenet-ssd** model

In [11]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs/mobilenet MYRIAD common/mobilenet-ssd/FP16/mobilenet-ssd.xml" -N obj_det_ncs2
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/ncs/mobilenet', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/ncs/mobilenet', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4752.v-qsvr-1.devcloud-edge


#### - Inferencing using model: ssd300

In [12]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs/ssd300 MYRIAD SSD300/FP16/ssd300.xml" -N obj_det_ncs2
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/ncs/ssd300', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/ncs/ssd300', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4754.v-qsvr-1.devcloud-edge


#### - Inferencing using model: ssd512

In [19]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc004nc2:intel-ncs2 -F "results/ncs/ssd512 MYRIAD SSD512/FP16/ssd512.xml" -N obj_det_ncs2
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/ncs/ssd512', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/ncs/ssd512', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4769.v-qsvr-1.devcloud-edge


### 4. Use the right data type for your target hardware and accuracy needs

In this section, we will consider an example running on a GPU. FP16 operations are better optimized than FP32 on GPUs. We will run the object detection example with SSD models with data types FP16 and FP32 and observe the performance difference.

In [16]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/GPU/mobilenet GPU common/mobilenet-ssd/FP32/mobilenet-ssd.xml" -N obj_det_gpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/GPU/mobilenet', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/mobilenet', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4760.v-qsvr-1.devcloud-edge


In [15]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl:intel-hd-530 -F "results/GPU/mobilenet GPU common/mobilenet-ssd/FP16/mobilenet-ssd.xml" -N obj_det_gpu
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/GPU/mobilenet', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/GPU/mobilenet', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)

4758.v-qsvr-1.devcloud-edge


It is clear that we got better performance with FP16 models.
### 5. Use async

The async API can improve the overall frame rate of the application. While the accelerator is busy with running inference operations, the application can continue encoding, decoding or post inference data processing on the host. For this section, we will use the object_detection_demo_ssd_async sample. This sample makes asynchronous requests to the inference engine. This reduces the inference request latency, so that the overall framerate is determined by the MAXIMUM(detection time, input capturing time) and not the SUM(detection time, input capturing time).

### a) Run the async example

In [12]:
!./object_detection_demo_ssd_async -i cars_1900.mp4 -m models/object_detection/common/mobilenet-ssd/FP32/mobilenet-ssd.xml

InferenceEngine: 
	API version ............ 2.1
	Build .................. custom_releases/2019/R3_cb6cad9663aea3d282e0e8b3e0bf359df665d5d0
	Description ....... API
[ INFO ] Parsing input parameters
[ INFO ] Reading input
[ INFO ] Loading plugin

	API version ............ 2.1
	Build .................. 30677
	Description ....... MKLDNNPlugin
[ INFO ] Loading network files
[ INFO ] Batch size is forced to  1.
[ INFO ] Checking that the inputs are as the demo expects
[ ERROR ] Data is empty!


Observe the number of fps (frames per second) for both sync and async mode in the videos below. The number frames processed per second are more in async than the sync mode.

There are important performance caveats though. Tasks that run in parallel should try to avoid oversubscribing to shared computing resources. For example, if the inference tasks are running on the FPGA and the CPU is essentially idle, then it makes sense to run tasks on the CPU in parallel.

In [ ]:
videoHTML('ASync Mode :',
           ['results/Core/output_async.mp4']
        )

In [18]:
!./object_detection_demo_ssd_sync -i cars_1900.mp4 -m models/object_detection/common/mobilenet-ssd/FP32/mobilenet-ssd.xml

InferenceEngine: 
	API version ............ 2.1
	Build .................. custom_releases/2019/R3_cb6cad9663aea3d282e0e8b3e0bf359df665d5d0
	Description ....... API
[ INFO ] Parsing input parameters
[ INFO ] Reading input
[ INFO ] Loading plugin

	API version ............ 2.1
	Build .................. 30677
	Description ....... MKLDNNPlugin
[ INFO ] Loading network files
[ INFO ] Batch size is forced to  1.
[ INFO ] Checking that the inputs are as the demo expects
[ ERROR ] Data is empty!
/opt/intel/openvino_2019.1.094/deployment_tools/inference_engine/include/ie_input_info.hpp:91


In [ ]:
videoHTML('Sync Mode :',
           ['results/Core/output_sync.mp4']
        )

In both the Video Sync and Async, we can see the inference timing. The timing for the async are lower and frames per second are higher than the sync Video